# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)<10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'llama3.2'
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
api_key

API key looks good so far


'sk-proj-'

In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())
ed.links

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [7]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [8]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-R1-Zero',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/webml-community/deepseek-r1-webgpu',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/cais/hle',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/yale-nlp/MMVU',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',

In [9]:
get_links("https://huggingface.co")

{'links': [{'type': 'About page', 'url': 'https://huggingface.co/'},
  {'type': 'Company page', 'url': 'https://huggingface.co/about'},
  {'type': 'Careers/Jobs page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'Blog page', 'url': 'https://blog.huggingface.co/'},
  {'type': 'Documentation page', 'url': 'https://huggingface.co/docs/'},
  {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter handle', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Discord community link',
   'url': 'https://join.discord.huggingface.co/'},
  {'type': 'Dataset documentation',
   'url': 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/chinese-language-blog/wechat.jpg'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [11]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': '', 'url': 'https://huggingface.co/'}, {'type': '', 'url': 'https://huggingface.co/brand'}, {'type': '', 'url': 'https://ui.endpoints.huggingface.co/chat'}, {'type': 'About page', 'url': 'https://ui.endpoints.huggingface.co/about'}, {'type': 'Company/Enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Prices/Plans page', 'url': 'https://huggingface.co/pricing'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': '', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the fu

In [12]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Amazon", "https://amazon.com")

Found links: {'links': [{'type': 'help page', 'url': 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou'}, {'type': 'privacy policy', 'url': 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy'}]}


"You are looking at a company called: Amazon\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nAmazon.com\nWebpage Contents:\nEnter the characters you see below\nSorry, we just need to make sure you're not a robot. For best results, please make sure your browser is accepting cookies.\nType the characters you see in this image:\nTry different image\nContinue shopping\nConditions of Use\nPrivacy Policy\n© 1996-2014, Amazon.com, Inc. or its affiliates\n\n\n\nhelp page\nWebpage Title:\n\n      Page Not Found\n    \nWebpage Contents:\n\n\n\n\nprivacy policy\nWebpage Title:\n\n      Page Not Found\n    \nWebpage Contents:\n\n\n"

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Amazon", "https://amazon.com")

Found links: {'links': [{'type': 'help and support', 'url': 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_cou?ie=UTF8&nodeId=508088'}, {'type': 'privacy policy', 'url': 'https://www.amazon.com/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=468496'}]}


# Welcome to Amazon

[Image of the Amazon logo]

## About Us

Amazon is a leading e-commerce platform that offers a wide range of products, services, and experiences for individuals, businesses, and organizations around the world. Since our humble beginnings in 1994 as an online bookstore, we have grown into one of the world's most influential and innovative companies.

## Our Mission

Our mission is to be the world's most customer-centric company, where customers can find and discover anything they might want to buy online. We aim to deliver exceptional customer experiences through our unparalleled selection, competitive prices, fast and reliable shipping, and outstanding customer service.

## Company Culture

At Amazon, we value innovation, diversity, and inclusion in everything we do. Our culture is built on a foundation of trust, respect, and open communication, where every employee feels empowered to make a difference. We are committed to creating a work environment that is inclusive, empowering, and focused on continuous learning and growth.

## Customers

We serve millions of customers across the globe, offering a vast array of products and services through our various business segments, including:

* Amazon.com: Our flagship consumer-facing platform
* Amazon Marketplace: A platform for third-party sellers to showcase their products
* Amazon Web Services (AWS): A comprehensive suite of cloud computing services

Our customers trust us with their personal and financial information, and we are committed to protecting their data with the highest level of security and integrity.

## Careers at Amazon

If you're passionate about innovation, customer obsession, and teamwork, we invite you to join our diverse and dynamic workforce. We offer a wide range of career opportunities across various fields, from technology and finance to marketing and logistics.

To learn more about our current openings and to apply, visit [our careers page](link).

## Get in Touch

Have questions or need assistance? Contact us at [feedback@amazon.com](mailto:feedback@amazon.com) or (866) 216-1076.

**Amazon is committed to making a positive impact on the world. Learn more about our sustainability initiatives and corporate social responsibility programs at [this link](link).**

---

[Image of the Amazon logo]

Join us on our mission to deliver unparalleled customer experiences and create innovative solutions that make a difference in people's lives.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("Amazon", "https://amazon.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.amazon.com/about/'}]}


# Amazon Brochure
## Company Overview

Welcome to Amazon, the world's leading e-commerce platform and innovative technology company. Founded in 1994 by Jeff Bezos, Amazon has grown from a small online bookstore to a global giant with over $1 trillion in annual revenue.

**Our Mission**

To be Earth's most customer-centric company, where customers can find and discover anything they might want to buy online.

## Company Culture

Amazon is known for its dynamic and fast-paced work environment. Our Leadership Principles guide our behavior and decision-making:

* Customer Obsession: We are passionate about delivering the best possible experience for our customers.
* Ownership: We take ownership of our work and commitments.
* Invent and Simplify: We believe in making a dent in the universe through innovation and simplicity.
* Are Right, A Lot: We strive to be right more often than not.
* Learn and Be Curious: We are curious and love to learn new things.
* Have Backbone but Stay Agile: We have the strength to make tough decisions but also adapt quickly to changing circumstances.
* Frugality: We live below our means and value efficiency in everything we do.
* Earn Trust: We build strong, lasting relationships with our customers, employees, and partners.
* Dive Deep: We prioritize complexity over simplicity.
* Have a Backseat: We empower our teams to make the right decisions without needing explicit approval.

## Customers

We are committed to delivering exceptional customer experiences across all our products and services:

* Amazon Fresh: Same-day delivery of fresh produce, meat, dairy products, and other household essentials.
* Prime Video: A vast library of original content, movies, TV shows, and apps.
* Prime Music: Access to a massive music streaming library.
* Alexa & Echo Devices: Smart speakers and displays that make it easy to control your smart home and access information.

## Careers/Jobs

Amazon offers various career opportunities across the globe:

* **Fulfillment Centers**: Working in our warehouses to receive, package, and send customer orders.
* **Customer Service Representatives**: Helping customers over the phone or through our chat platform.
* **Software Engineers**: Developing new technologies that power our products and services.
* **Marketing & Sales**: Driving business growth and promoting Amazon's products and services.

**Small Businesses**

We support small businesses across various industries:

* Amazon Merch: Sell your designs on our marketplace.
* Fulfillment by Merchant (FBM): Partner with us to ship products and access customer information.
* Third-Party Sellers: Expand your online presence by selling through Amazon.

## Sustainability

Amazon is committed to creating a more sustainable future:

* **Renewable Energy**: Our carbon-free energy strategy aims to power all our operations with renewable energy sources.
* **Sustainable Packaging**: We're working to reduce waste and make packaging more environmentally friendly.
* **Climate Pledge**: A bold challenge to reduce Amazon's carbon footprint to net-zero by 2040.

Join us at Amazon, where innovation meets customer obsession. Explore our careers, products, and services today!

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("LeetCode","https://leetcode.com")

Found links: {'links': [{'type': 'company webpage', 'url': 'https://leetcode.com'}, {'type': 'careers webpage', 'url': 'https://leetcode.com/career-stories/'}, {'type': 'about webpage', 'url': 'https://leetcode.com/blog/'}, {'type': 'team webpage', 'url': 'https://leetcode.com/team/'}, {'type': 'careers/jobs webpage', 'url': 'https://leetcode.com/careers/'}]}


# LeetCode Brochure
=====================

LeetCode is more than just a platform - it's a community of passionate developers who come together to learn, grow, and push the boundaries of coding excellence.

## Our Mission
-----------------

At LeetCode, our mission is to provide a comprehensive and inclusive platform for learning and practicing programming skills. We strive to inspire a love of coding in individuals from all walks of life, regardless of their background or experience level.

## Company Culture
------------------

Our culture is built on the principles of collaboration, innovation, and inclusivity. We believe that every individual has the potential to become an exceptional developer, and we're committed to creating an environment that fosters growth, learning, and fun. Here are some values that define our company culture:

* **Learn from each other**: Our community is built on the idea of mutual support and knowledge sharing.
* **Get hands-on experience**: We believe that coding skills can't be learned through theory alone - you need to practice!
* **Stay curious**: We encourage our users and team members to stay curious and keep learning.

## What We Do
--------------

LeetCode is an online platform that offers a vast collection of coding challenges, interview practice problems, and resources for developers. Our platform is designed to help developers improve their coding skills, expand their knowledge, and land jobs at top tech companies.

### Features

* **Millions of user-generated problem sets**: LeetCode has one of the largest and most comprehensive collections of coding challenges in the world.
* **Interview practice problems**: We partner with top tech companies to provide interview practice problems for developers.
* **Community forum**: Our community forum is where users can share their knowledge, get help from experts, and build meaningful connections.

## Our Community
-----------------

LeetCode has a vibrant community of over 50 million registered users worldwide. Our users come from all backgrounds - beginners, experts, hobbyists, professionals, and companies alike. We believe that our community is the backbone of our platform, and we're committed to creating an experience that's fun, engaging, and rewarding.

### Customers

* **Individual developers**: LeetCode is the number one destination for individual developers who want to improve their coding skills.
* **Companies**: We partner with top tech companies to provide interview practice problems and resources for hiring.
* **Education institutions**: Our platform is used by schools, universities, and online course providers to support education and skill-building.

## Careers/Jobs
----------------

Are you passionate about coding and looking for a new opportunity? LeetCode offers a range of exciting jobs and internships that are perfect for both beginners and experienced developers. Here are some ways you can join our team:

* **Software Engineer**: Help us build the next generation of coding challenges and interview practice problems.
* **Technical Writer**: Write clear, concise, and engaging content about coding concepts and best practices.
* **Quality Assurance Specialist**: Ensure that our platform is fun, functional, and error-free.

### How to Apply

Check out our careers webpage for more details on our current job openings. If you're passionate about learning, growing, and pushing the boundaries of coding excellence - we want to hear from you!

[Learn more](link to careers webpage)

We look forward to welcoming new team members into our vibrant community!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>